# The asyncio Module

According to Python's documentation the asyncio module "provides infrastructure for writing single-threaded concurrent code using coroutines, multiplexing I/O access over sockets and other resources, running network clients and servers, and other related primitives“.

## Getting Started

Asyncio is built around the event loop. An event loops waits for something to happen then acts on the event. It handles I/O and ssystem events. An event loop basically says when A happenns execute B.

Another important aspect of asyncio is the coroutine which is a special function that gives control up to the caller without losing its statre. One of the big benefiots over threads is that they dont use much memory. When you call a coroutine function it doesn't execute right away, instead it will return a coroutine object that you can pass to the event loop to have it executed either immediately or later on.

A future is basically an object that represents the result of work that hasn’t completed. A Task is a wrapper for a coroutine and a subclass of Future.

The use of coroutines can speed up your python code significantly by starting on new executions while waiting for previous ones to return values.

## async and await

Starting in Python 3.5 you can create coroutines with native syntac using the async and await special keywords.

In [5]:
# Simple coroutine example
# The async / await keywords can be considered an API to be used for asynchronous programming.
import asyncio

async def my_coroutine():
    await func()

## A Bad Coroutine Example

In [10]:
# using async to download files
# this is much faster than calling sequentially
# Have to use different syntax from example to make this compatible with jupyter notebooks
# jupyter runtimes have a default event loop running so no need to create one

import asyncio
import os
import urllib.request

async def download_coroutine(url):
    """
    A coroutine to download the specified url
    """
    request = urllib.request.urlopen(url)
    filename = os.path.basename(url)

    with open(filename, 'wb') as file_handle:
        while True:
            chunk = request.read(1024)
            if not chunk:
                break
            file_handle.write(chunk)
    msg = 'Finished downloading {filename}'.format(filename=filename)
    return msg

async def main(urls):
    """
    Creates a group of coroutines and waits for them to finish
    """
    coroutines = [download_coroutine(url) for url in urls]
    completed, pending = await asyncio.wait(coroutines)
    for item in completed:
        print(item.result())


if __name__ == '__main__':
    urls = ["http://www.irs.gov/pub/irs-pdf/f1040.pdf",
            "http://www.irs.gov/pub/irs-pdf/f1040a.pdf",
            "http://www.irs.gov/pub/irs-pdf/f1040ez.pdf",
            "http://www.irs.gov/pub/irs-pdf/f1040es.pdf",
            "http://www.irs.gov/pub/irs-pdf/f1040sb.pdf"]

    await main(urls)

Finished downloading f1040sb.pdf
Finished downloading f1040.pdf
Finished downloading f1040ez.pdf
Finished downloading f1040a.pdf
Finished downloading f1040es.pdf


Since the urllib package is not asynchronous this is not event a real coroutine at all. A better way to do this would be to use the aiohttp package. Let’s look at that next!

## A Better Coroutine Example

In [11]:
# Install aiohttp for creating async http clients and servers
!pip install aiohttp

In [20]:
# Modified code from above using aiohttp
# You can see we use await ketyword in our download_coroutine function now
# We also set up our client with a context manager so it breaks down after completing the job
import aiohttp
import asyncio
import async_timeout
import os


async def download_coroutine(session, url):
    with async_timeout.timeout(100):
        async with session.get(url) as response:
            filename = os.path.basename(url)
            with open(filename, 'wb') as f_handle:
                while True:
                    chunk = await response.content.read(1024)
                    if not chunk:
                        break
                    f_handle.write(chunk)
            return await response.release()


async def main(loop):
    urls = ["http://www.irs.gov/pub/irs-pdf/f1040.pdf",
            "http://www.irs.gov/pub/irs-pdf/f1040a.pdf",
            "http://www.irs.gov/pub/irs-pdf/f1040ez.pdf",
            "http://www.irs.gov/pub/irs-pdf/f1040es.pdf",
            "http://www.irs.gov/pub/irs-pdf/f1040sb.pdf"]

    async with aiohttp.ClientSession(loop=loop) as session:
        for url in urls:
            print(await download_coroutine(session, url))


if __name__ == '__main__':
    await main(loop)

None
None
None
None
None


## Scheduling Calls

The call_soon method will basically call your callback or event handler as soon as it can. It works as a FIFO queue, so if some of the callbacks take a while to run, then the others will be delayed until the previous ones have finished. 

In [3]:
import asyncio
import functools

# This syntax won't execute properly in a jupyter notebook due to running event loop
# Although this is how you would schedule calls in a .py file

def event_handler(loop, stop=False):
    print('Event handler called')
    if stop:
        print('stopping the loop')
        loop.stop()


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    try:
        loop.call_soon(functools.partial(event_handler, loop))
        print('starting event loop')
        loop.call_soon(functools.partial(event_handler, loop, stop=True))

        loop.run_forever()
    finally:
        print('closing event loop')

## Tasks

Tasks are a subclass of a Future and a wrapper around a coroutine. They give you the ability to keep track of when they finish processing.

In [ ]:
import asyncio

# This syntax won't execute properly in a jupyter notebook due to running event loop
# Although this is how you would schedule calls in a .py file

async def my_task(seconds):
    """
    A task to do for a number of seconds
    """
    print('This task is taking {} seconds to complete'.format(
        seconds))
    asyncio.sleep(seconds)
    return 'task finished'


if __name__ == '__main__':
    my_event_loop = asyncio.get_event_loop()
    try:
        print('task creation started')
        task_obj = my_event_loop.create_task(my_task(seconds=2))
        my_event_loop.run_until_complete(task_obj)
    finally:
        my_event_loop.close()

    print("The task's result was: {}".format(task_obj.result()))